<a href="https://colab.research.google.com/github/once-upon-an-april/Thuc-Hanh-Deep-Learning-trong-Khoa-Hoc-Du-Lieu-DS201.Q11.1/blob/main/Bai2/22520975_Lab3_Bai1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# BÀI 1: PHÂN LOẠI CẢM XÚC TRÊN DATASET UIT-VSFC SỬ DỤNG LSTM (5 LỚP, HIDDEN 256)
# ==============================================================================

# 1. CÀI ĐẶT THƯ VIỆN CẦN THIẾT
!pip install pyvi datasets torch scikit-learn

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score, classification_report
from datasets import load_dataset
from pyvi import ViTokenizer

# Kiểm tra GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# ==============================================================================
# PHẦN 1: TẢI VÀ TIỀN XỬ LÝ DỮ LIỆU (PREPROCESSING)
# ==============================================================================

print("\n>>> 1. Đang tải dataset UIT-VSFC...")
# Tải phiên bản Parquet để tránh lỗi script bảo mật
try:
    ds = load_dataset("uitnlp/vietnamese_students_feedback", name="default", revision="refs/convert/parquet")
except Exception:
    # Fallback nếu mạng lỗi
    ds = load_dataset("uitnlp/vietnamese_students_feedback", trust_remote_code=True)

# Chuyển sang Pandas DataFrame
train_df = pd.DataFrame(ds['train'])
val_df = pd.DataFrame(ds['validation'])
test_df = pd.DataFrame(ds['test'])

# --- HÀM TIỀN XỬ LÝ ---
def preprocess_text(text):
    # 1. Chuyển về chữ thường
    text = text.lower()
    # 2. Loại bỏ dấu câu (giữ lại tiếng Việt và số)
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    # 3. Tách từ tiếng Việt
    text = ViTokenizer.tokenize(text)
    return text

print(">>> Đang thực hiện Preprocessing (Clean & Tokenize)...")
train_df['sentence_cleaned'] = train_df['sentence'].astype(str).apply(preprocess_text)
val_df['sentence_cleaned'] = val_df['sentence'].astype(str).apply(preprocess_text)
test_df['sentence_cleaned'] = test_df['sentence'].astype(str).apply(preprocess_text)

print("Mẫu dữ liệu sau khi làm sạch:")
print(train_df[['sentence', 'sentence_cleaned']].head(2))


In [ ]:
# ==============================================================================
# PHẦN 2: XÂY DỰNG TỪ ĐIỂN VÀ DATASET CHO LSTM
# (Thay thế TF-IDF bằng Word Indexing cho mô hình Sequence)
# ==============================================================================

# Cấu hình Hyperparameters
MAX_LEN = 50       # Độ dài tối đa của câu (padding/truncate)
BATCH_SIZE = 64    # Kích thước batch
EMBEDDING_DIM = 100 # Kích thước vector biểu diễn từ
HIDDEN_SIZE = 256  # Yêu cầu đề bài
NUM_LAYERS = 5     # Yêu cầu đề bài: 5 lớp LSTM
NUM_CLASSES = 3    # Negative, Neutral, Positive
LEARNING_RATE = 0.001
EPOCHS = 10

# 1. Xây dựng bộ từ điển (Vocabulary)
print("\n>>> 2. Xây dựng Vocabulary...")
all_words = " ".join(train_df['sentence_cleaned']).split()
word_counts = Counter(all_words)

# Chỉ giữ lại các từ xuất hiện ít nhất 2 lần để giảm nhiễu
vocab = sorted([w for w, c in word_counts.items() if c >= 2])
word2idx = {w: i+2 for i, w in enumerate(vocab)} # i+2 vì 0 dành cho padding, 1 cho unknown
word2idx['<PAD>'] = 0
word2idx['<UNK>'] = 1

vocab_size = len(word2idx)
print(f"Kích thước từ điển (Vocab Size): {vocab_size}")

# 2. Hàm chuyển câu văn bản thành chuỗi số (Vectorize)
def vectorize_text(text, word2idx, max_len):
    words = text.split()
    # Chuyển từ thành index, nếu không có thì dùng <UNK>
    idxs = [word2idx.get(w, word2idx['<UNK>']) for w in words]

    # Padding hoặc Truncate
    if len(idxs) < max_len:
        idxs = idxs + [word2idx['<PAD>']] * (max_len - len(idxs))
    else:
        idxs = idxs[:max_len]
    return idxs

# 3. Tạo Custom Dataset cho PyTorch
class VSFC_Dataset(Dataset):
    def __init__(self, df, word2idx, max_len):
        self.sentences = df['sentence_cleaned'].values
        self.labels = df['sentiment'].values
        self.word2idx = word2idx
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        text = self.sentences[idx]
        label = self.labels[idx]

        # Vector hóa
        vector = vectorize_text(text, self.word2idx, self.max_len)

        return torch.tensor(vector, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# Tạo DataLoader
train_dataset = VSFC_Dataset(train_df, word2idx, MAX_LEN)
val_dataset = VSFC_Dataset(val_df, word2idx, MAX_LEN)
test_dataset = VSFC_Dataset(test_df, word2idx, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
# ==============================================================================
# PHẦN 3: ĐỊNH NGHĨA MÔ HÌNH LSTM (5 LỚP, HIDDEN 256)
# ==============================================================================

class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_dim, n_layers):
        super(SentimentLSTM, self).__init__()

        # 1. Embedding Layer: Chuyển index thành vector dày đặc
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # 2. LSTM Layers
        # input_size = embedding_dim
        # batch_first=True: input shape là (batch, seq_len, features)
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=n_layers,
            batch_first=True,
            dropout=0.3  # Thêm dropout để tránh Overfitting vì model khá sâu (5 lớp)
        )

        # 3. Fully Connected Layer (Output)
        self.fc = nn.Linear(hidden_size, output_dim)

    def forward(self, text):
        # text shape: [batch size, sent len]

        embedded = self.embedding(text)
        # embedded shape: [batch size, sent len, emb dim]

        # LSTM output
        # output: chứa hidden state của tất cả time steps
        # (hidden, cell): trạng thái ẩn cuối cùng
        output, (hidden, cell) = self.lstm(embedded)

        # Lấy hidden state của lớp cuối cùng (layer cuối, time step cuối)
        # hidden shape: [num_layers, batch size, hidden dim]
        # Ta lấy lớp cuối cùng: hidden[-1, :, :]
        last_hidden = hidden[-1, :, :]

        return self.fc(last_hidden)

# Khởi tạo mô hình
model = SentimentLSTM(vocab_size, EMBEDDING_DIM, HIDDEN_SIZE, NUM_CLASSES, NUM_LAYERS)
model = model.to(device)

print("\n>>> 3. Kiến trúc mô hình:")
print(model)

# Hàm Loss và Optimizer (Adam)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# ==============================================================================
# PHẦN 4: HUẤN LUYỆN VÀ ĐÁNH GIÁ (TRAINING LOOP)
# ==============================================================================

def calculate_metrics(y_true, y_pred):
    # Average='weighted' phù hợp cho dataset mất cân bằng
    f1 = f1_score(y_true, y_pred, average='weighted')
    return f1

print("\n>>> 4. Bắt đầu huấn luyện...")

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    all_preds = []
    all_labels = []

    for text, labels in train_loader:
        text, labels = text.to(device), labels.to(device)

        optimizer.zero_grad()
        predictions = model(text)

        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Lưu lại để tính metrics
        preds = torch.argmax(predictions, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

    train_f1 = calculate_metrics(all_labels, all_preds)

    # --- Validation ---
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for text, labels in val_loader:
            text, labels = text.to(device), labels.to(device)
            predictions = model(text)
            loss = criterion(predictions, labels)
            val_loss += loss.item()

            preds = torch.argmax(predictions, dim=1).cpu().numpy()
            val_preds.extend(preds)
            val_labels.extend(labels.cpu().numpy())

    val_f1 = calculate_metrics(val_labels, val_preds)

    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss/len(train_loader):.4f} | Train F1: {train_f1:.4f} | "
          f"Val Loss: {val_loss/len(val_loader):.4f} | Val F1: {val_f1:.4f}")

In [ ]:
# ==============================================================================
# PHẦN 5: ĐÁNH GIÁ TRÊN TẬP TEST
# ==============================================================================
print("\n>>> 5. Đánh giá kết quả trên tập TEST...")
model.eval()
test_preds = []
test_labels = []

with torch.no_grad():
    for text, labels in test_loader:
        text, labels = text.to(device), labels.to(device)
        predictions = model(text)
        preds = torch.argmax(predictions, dim=1).cpu().numpy()
        test_preds.extend(preds)
        test_labels.extend(labels.cpu().numpy())

# In báo cáo chi tiết
print("\nClassification Report (Test Set):")
target_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(test_labels, test_preds, target_names=target_names, digits=4))

f1_final = f1_score(test_labels, test_preds, average='weighted')
print(f"Final Weighted F1-Score: {f1_final:.4f}")